In [1]:
from config.configs import get_config
from utils.modellib import getModel, loadModel
from utils.datalib import getDataset, splitDataset
from utils.toollib import ToTensor, ToNumpy

import torch
import torch.nn as nn
import numpy as np
import os
from tqdm import tqdm
from termcolor import colored

import sklearn
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans

from scipy.optimize import linear_sum_assignment

def getFeatures(net, loader):
    features = []
    labels =[]
    net.eval()
    
    for batch in tqdm(loader):
        nodes = ToTensor(batch['node']).cuda()
        label = ToNumpy(batch['label'])
        labels.append(label)
        with torch.no_grad():
            fea = net.backbone(nodes)
            fea = fea.cpu().numpy()
        features.append(fea)
        
    features = np.concatenate(features, axis=0)
    labels = np.concatenate(labels, axis=0)
    print('feature shape: {}, labels shape: {}'.format(features.shape, labels.shape))
    return features, labels

def getPretrainModelandFeatures(task='supervised', model_type='PointSwin', point2img=False, opti='adamw', load_pretrain=True):
    assert task in ['supervised', 'simclr', 'byol', 'simsiam', 'moco']
    
    config = get_config(task=task, model_type=model_type, point2img=point2img, opti=opti)
    train_config = config.train_config
    paths = config.path_config
    model_config = config.model_config
    data_config = config.data_config
    
    data_config['task'] = 'supervised'
    data_config['one_hot']=False
    
    epoch_start=0
    # print cfg
    print(colored('Info Config:', 'red'))
    print(colored(config.info_config, 'red'))
    print(colored('Backbone Config:', 'red'))
    print(colored(model_config, 'red'))

    # create and load model
    print(colored('Get backbone', 'blue'))
    net = getModel(config)
    if load_pretrain:
        last_model_file = os.path.join(config.path_config['{}_checkpoint'.format(task)],'{}_last_epoch.pth'.format(task+model_type))
        if os.path.exists(last_model_file):
            net, epoch_start = loadModel(net=net,
                                         save_path=paths['{}_checkpoint'.format(task)],
                                         file_class='last',
                                         model_name=task+model_type, 
                                         task=task)
            print(colored('Restart from checkpoint {}'.format(paths['{}_dir'.format(task)]), 'blue'))
        else:
            print(colored('Connot found any pre-train model', 'yellow'))
    else:
        print('Random init model')
    # create dataset
    dataset = getDataset(data_config)
    if os.path.exists(paths['train_index']) and os.path.exists(paths['val_index']):
        splited_out = splitDataset(dataset=dataset, cfg=config, use_pretrain_indexes=True)
    else:
        splited_out = splitDataset(dataset=dataset, cfg=config)
    train_loader, val_loader = splited_out['train_dataloader'], splited_out['val_dataloader']
    train_num, val_num = splited_out['train_num'], splited_out['val_num']
    print(colored('Get dataset and dataloaders', 'blue'))

    device = train_config['device']
    net = net.to(device=device)
    
    print('get train features:')
    train_feas, train_labels = getFeatures(net=net, loader=train_loader)
    print('get val features:')
    val_feas, val_labels = getFeatures(net=net, loader=val_loader)
    
    return net, train_feas, train_labels, val_feas, val_labels

def getConfusionMatrix(pred, target):
    pred_cls = np.unique(pred)
    targer_cls = np.unique(target)
    clusters = max(len(pred_cls), len(targer_cls))
    CMatrix = np.zeros((clusters, clusters))
    for index in range(len(target)):
        i,j = target[index], pred[index]
        CMatrix[i,j]+=1
    #print(CMatrix)
    return CMatrix

def KMeans_train_and_predict(ncluster, train_fea, val_fea):
    print('training Kmeans model...')
    KmeansModel = KMeans(n_clusters=ncluster, max_iter=10000)
    KmeansModel.fit(train_fea)
    
    y_pred_train = KmeansModel.predict(train_fea)
    y_pred_val = KmeansModel.predict(val_fea)
    
    return KmeansModel, y_pred_train, y_pred_val

def getMatch(pred, label):
    assert len(pred)==len(label)
    cmx = getConfusionMatrix(pred=pred, target=label)
    cost = len(label)-cmx
    matches = linear_sum_assignment(cost)
    
    return matches

def assignPred(matches, pred):
    assign_y_pred = np.zeros_like(pred)
    for i in range(len(matches[1])):
        assign_y_pred[pred==matches[1][i]] = i
    return assign_y_pred

def evaluate_result(features, labels, assign_y_pred):
    result_NMI=metrics.normalized_mutual_info_score(labels, assign_y_pred)
    result_ARI = metrics.adjusted_rand_score(labels, assign_y_pred)
    result_ACC = metrics.accuracy_score(labels, assign_y_pred)
    result_lab_CH = metrics.calinski_harabasz_score(features, labels)
    result_lab_SC = metrics.silhouette_score(features, labels)
    result_pred_CH = metrics.calinski_harabasz_score(features, assign_y_pred)
    result_pred_SC = metrics.silhouette_score(features, assign_y_pred)
    res = {'NMI':result_NMI, 'ARI':result_ARI, 'ACC':result_ACC, 
           'labCH':result_lab_CH, 'labSC':result_lab_SC, 
           'predCH':result_pred_CH, 'predSC':result_pred_SC}
    return res

def train_and_val_Kmeans(ncluster, train_feas, train_labels, val_feas, val_labels):
    KmeansModel, y_pred_train, y_pred_val = KMeans_train_and_predict(ncluster=ncluster, train_fea=train_feas, val_fea=val_feas)
    print('get matches')
    matches = getMatch(pred=y_pred_train, label=train_labels)
    print('Assigning pred...')
    assignYtrain = assignPred(matches=matches, pred=y_pred_train)
    assignYval = assignPred(matches=matches, pred=y_pred_val)
    print('Counting confusion matrix...')
    Tcmx = getConfusionMatrix(pred=assignYtrain, target=train_labels)
    Vcmx = getConfusionMatrix(pred=assignYval, target=val_labels)
    print('train confusion matrix:')
    print(Tcmx)
    print('val confusion matrix')
    print(Vcmx)
    
    train_res = evaluate_result(features=train_feas,labels=train_labels, assign_y_pred=assignYtrain)
    val_res = evaluate_result(features=val_feas, labels=val_labels, assign_y_pred=assignYval)
    print('train result:')
    print(train_res)
    print('val result:')
    print(val_res)
    
    return {'kmeans':KmeansModel, 'tcmx':Tcmx, 'vcmx':Vcmx, 'train_res':train_res, 'val_res':val_res, 'matches':matches}

def train_pretextKmeans(ncluster, task='supervised', model_type='PointSwin', point2img=False, opti='adamw'):
    net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, model_type=model_type, point2img=point2img, opti=opti)
    kmeans_result = train_and_val_Kmeans(ncluster, train_feas, train_labels, val_feas, val_labels)

In [2]:
clusters_num = [2,3,4,5,6]

In [3]:
task='supervised'
model_type='PointSwin'
point2img=False
opti='adamw'
net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, 
                                                                                  model_type=model_type, 
                                                                                  point2img=point2img, 
                                                                                  opti=opti)
for clsnum in clusters_num:
    print('='*20)
    print('cluster num: {}'.format(clsnum))
    kmeans_result = train_and_val_Kmeans(clsnum, train_feas, train_labels, val_feas, val_labels)

Info Config:
{'dataset_path': './data', 'result_root': './result/Clustering', 'task': 'supervised', 'train_db_name': 'tube', 'val_db_name': 'tube', 'BackboneType': 'PointSwin'}
Backbone Config:
{'BackboneConfig': {'in_channels': 15, 'feature_dim': 128, 'hiddim': 64, 'heads': 8, 'headdim': 32, 'embeddim': 4, 'stage_num': 3, 'downscale': 4, 'layer_num': 1, 'window_size': 4, 'attnlayer': 1, 'num_class': 4}, 'BackboneType': 'PointSwin', 'task': 'supervised', 'nheads': 1}
Get backbone


  0%|                                                                                          | 0/295 [00:00<?, ?it/s]

Restart from checkpoint ./result/Clustering\tube\PointSwin\supervised\
Get dataset and dataloaders
get train features:


  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

feature shape: (2360, 128), labels shape: (2360,)
get val features:


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:24<00:00,  3.01it/s]


feature shape: (590, 128), labels shape: (590,)
cluster num: 2
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[  0. 418.   0.  61.]
 [  0. 647.   0.   9.]
 [  0. 389.   0.   4.]
 [  0.  36.   0. 796.]]
val confusion matrix
[[  0. 101.   0.  20.]
 [  0. 151.   0.   3.]
 [  0. 101.   0.   1.]
 [  0.   9.   0. 204.]]
train result:
{'NMI': 0.48747223866647166, 'ARI': 0.4021498086429194, 'ACC': 0.6114406779661017, 'labCH': 1241.8535033350083, 'labSC': 0.38339654, 'predCH': 1079.6593236091217, 'predSC': 0.31671405}
val result:
{'NMI': 0.4741109163226885, 'ARI': 0.40067580938952874, 'ACC': 0.6016949152542372, 'labCH': 234.38076668905887, 'labSC': 0.2996083, 'predCH': 275.1863849606992, 'predSC': 0.31180486}
cluster num: 3
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[424.  14.   0.  41.]
 [  0. 648.   0.   8.]
 [ 13. 376.   0.   4.]
 [  7.  33.   0. 792.]]
val confusio

In [3]:
task='supervised'
model_type='PointTrans'
point2img=False
opti='adamw'
net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, 
                                                                                  model_type=model_type, 
                                                                                  point2img=point2img, 
                                                                                  opti=opti)
for clsnum in clusters_num:
    print('='*20)
    print('cluster num: {}'.format(clsnum))
    kmeans_result = train_and_val_Kmeans(clsnum, train_feas, train_labels, val_feas, val_labels)

Info Config:
{'dataset_path': './data', 'result_root': './result/Clustering', 'task': 'supervised', 'train_db_name': 'tube', 'val_db_name': 'tube', 'BackboneType': 'PointTrans'}
Backbone Config:
{'BackboneConfig': {'in_channels': 15, 'feature_dim': 128, 'embedding_dim': 32, 'npoints': 1024, 'nblocks': 4, 'nneighbor': 32, 'transformer_dim': 128, 'num_class': 4}, 'BackboneType': 'PointTrans', 'task': 'supervised', 'nheads': 1}
Get backbone


  0%|                                                                                          | 0/295 [00:00<?, ?it/s]

Restart from checkpoint ./result/Clustering\tube\PointTrans\supervised\
Get dataset and dataloaders
get train features:


  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

feature shape: (2360, 128), labels shape: (2360,)
get val features:


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:19<00:00,  3.76it/s]


feature shape: (590, 128), labels shape: (590,)
cluster num: 2
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[  0. 449.   0.  31.]
 [  0. 645.   0.   3.]
 [  0. 397.   0.   0.]
 [  0.  17.   0. 818.]]
val confusion matrix
[[  0. 109.   0.  11.]
 [  0. 162.   0.   0.]
 [  0.  98.   0.   0.]
 [  0.   9.   0. 201.]]
train result:
{'NMI': 0.561949303880347, 'ARI': 0.4400335624663568, 'ACC': 0.6199152542372881, 'labCH': 1563.48702389912, 'labSC': 0.4555455, 'predCH': 1105.053622470101, 'predSC': 0.3161716}
val result:
{'NMI': 0.5283462827768738, 'ARI': 0.41370036493853274, 'ACC': 0.6152542372881356, 'labCH': 316.0621828696903, 'labSC': 0.3907466, 'predCH': 291.64668592831805, 'predSC': 0.320515}
cluster num: 3
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[464.   0.   0.  16.]
 [  6. 638.   0.   4.]
 [382.  15.   0.   0.]
 [ 16.   5.   0. 814.]]
val confusion matrix


In [3]:
task='simsiam'
model_type='PointTrans'
point2img=False
opti='adamw'
net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, 
                                                                                  model_type=model_type, 
                                                                                  point2img=point2img, 
                                                                                  opti=opti)
for clsnum in clusters_num:
    print('='*20)
    print('cluster num: {}'.format(clsnum))
    kmeans_result = train_and_val_Kmeans(clsnum, train_feas, train_labels, val_feas, val_labels)

Info Config:
{'dataset_path': './data', 'result_root': './result/Clustering', 'task': 'simsiam', 'train_db_name': 'tube', 'val_db_name': 'tube', 'BackboneType': 'PointTrans'}
Backbone Config:
{'BackboneConfig': {'in_channels': 15, 'feature_dim': 128, 'embedding_dim': 32, 'npoints': 1024, 'nblocks': 4, 'nneighbor': 32, 'transformer_dim': 128}, 'BackboneType': 'PointTrans', 'num_cluster': 4, 'task': 'simsiam', 'head': 'mlp', 'nheads': 1, 'contrastive_feadim': 128}
Get backbone


  0%|                                                                                          | 0/295 [00:00<?, ?it/s]

Restart from checkpoint ./result/Clustering\tube\PointTrans\pretext\simsiam\simsiam\
Get dataset and dataloaders
get train features:


  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

feature shape: (2360, 128), labels shape: (2360,)
get val features:


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:18<00:00,  3.90it/s]


feature shape: (590, 128), labels shape: (590,)
cluster num: 2
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[ 60.   0.   0. 416.]
 [ 53.   0.   0. 607.]
 [ 41.   0.   0. 351.]
 [ 86.   0.   0. 746.]]
val confusion matrix
[[ 11.   0.   0. 113.]
 [  8.   0.   0. 142.]
 [ 12.   0.   0.  91.]
 [ 25.   0.   0. 188.]]
train result:
{'NMI': 0.0016437972644385832, 'ARI': 0.002148888305956698, 'ACC': 0.34152542372881356, 'labCH': 15.571900144587644, 'labSC': -0.07898046, 'predCH': 7001.549466209627, 'predSC': 0.868564}
val result:
{'NMI': 0.005390612532870912, 'ARI': -0.002567801523085453, 'ACC': 0.33728813559322035, 'labCH': 3.3297815730959415, 'labSC': -0.06944669, 'predCH': 2211.5418229440847, 'predSC': 0.8843598}
cluster num: 3
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[ 60.   1.   0. 415.]
 [ 51.  98.   0. 511.]
 [ 40.  29.   0. 323.]
 [ 83.   3.   0. 746.]]
va

In [3]:
task='simsiam'
model_type='PointSwin'
point2img=False
opti='adamw'
net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, 
                                                                                  model_type=model_type, 
                                                                                  point2img=point2img, 
                                                                                  opti=opti)
for clsnum in clusters_num:
    print('='*20)
    print('cluster num: {}'.format(clsnum))
    kmeans_result = train_and_val_Kmeans(clsnum, train_feas, train_labels, val_feas, val_labels)

Info Config:
{'dataset_path': './data', 'result_root': './result/Clustering', 'task': 'simsiam', 'train_db_name': 'tube', 'val_db_name': 'tube', 'BackboneType': 'PointSwin'}
Backbone Config:
{'BackboneConfig': {'in_channels': 15, 'feature_dim': 128, 'hiddim': 64, 'heads': 8, 'headdim': 32, 'embeddim': 4, 'stage_num': 3, 'downscale': 4, 'layer_num': 1, 'window_size': 4, 'attnlayer': 1}, 'BackboneType': 'PointSwin', 'num_cluster': 4, 'task': 'simsiam', 'head': 'mlp', 'nheads': 1, 'contrastive_feadim': 128}
Get backbone


  0%|                                                                                          | 0/295 [00:00<?, ?it/s]

Restart from checkpoint ./result/Clustering\tube\PointSwin\pretext\simsiam\simsiam\
Get dataset and dataloaders
get train features:


  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

feature shape: (2360, 128), labels shape: (2360,)
get val features:


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:10<00:00,  7.16it/s]


feature shape: (590, 128), labels shape: (590,)
cluster num: 2
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[  0.   4.   0. 480.]
 [  0. 162.   0. 492.]
 [  0.  53.   0. 341.]
 [  0.   8.   0. 820.]]
val confusion matrix
[[  0.   0.   0. 116.]
 [  0.  38.   0. 118.]
 [  0.  20.   0.  81.]
 [  0.   0.   0. 217.]]
train result:
{'NMI': 0.08012938998105447, 'ARI': 0.026779063366221226, 'ACC': 0.4161016949152542, 'labCH': 79.87043903506877, 'labSC': -0.110952914, 'predCH': 1823.1087630711424, 'predSC': 0.7013079}
val result:
{'NMI': 0.10747393499782311, 'ARI': 0.04066599800862217, 'ACC': 0.4322033898305085, 'labCH': 22.09497600240662, 'labSC': -0.092039295, 'predCH': 481.8016993741955, 'predSC': 0.70583993}
cluster num: 3
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[ 60.   4.   0. 420.]
 [ 58. 157.   0. 439.]
 [ 46.  49.   0. 299.]
 [ 98.   7.   0. 723.]]
val con

In [3]:
task='supervised'
model_type='PointSwin'
point2img=False
opti='adamw'
net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, 
                                                                                  model_type=model_type, 
                                                                                  point2img=point2img, 
                                                                                  opti=opti,
                                                                                  load_pretrain=False)
for clsnum in clusters_num:
    print('='*20)
    print('cluster num: {}'.format(clsnum))
    kmeans_result = train_and_val_Kmeans(clsnum, train_feas, train_labels, val_feas, val_labels)

  0%|                                                                                          | 0/295 [00:00<?, ?it/s]

Info Config:
{'dataset_path': './data', 'result_root': './result/Clustering', 'task': 'supervised', 'train_db_name': 'tube', 'val_db_name': 'tube', 'BackboneType': 'PointSwin'}
Backbone Config:
{'BackboneConfig': {'in_channels': 15, 'feature_dim': 128, 'hiddim': 64, 'heads': 8, 'headdim': 32, 'embeddim': 4, 'stage_num': 3, 'downscale': 4, 'layer_num': 1, 'window_size': 4, 'attnlayer': 1, 'num_class': 4}, 'BackboneType': 'PointSwin', 'task': 'supervised', 'nheads': 1}
Get backbone
Random init model
Get dataset and dataloaders
get train features:


  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

feature shape: (2360, 128), labels shape: (2360,)
get val features:


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:12<00:00,  5.73it/s]


feature shape: (590, 128), labels shape: (590,)
cluster num: 2
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[  0. 294.   0. 185.]
 [  0. 319.   0. 337.]
 [  0. 194.   0. 199.]
 [  0. 309.   0. 523.]]
val confusion matrix
[[  0.  77.   0.  44.]
 [  0.  85.   0.  69.]
 [  0.  57.   0.  45.]
 [  0.  80.   0. 133.]]
train result:
{'NMI': 0.015453636732055524, 'ARI': 0.01861502824301428, 'ACC': 0.35677966101694913, 'labCH': 18.484698013772995, 'labSC': -0.0074504963, 'predCH': 332.3694003392451, 'predSC': 0.10968902}
val result:
{'NMI': 0.021117146418852338, 'ARI': 0.02217073695946048, 'ACC': 0.3694915254237288, 'labCH': 6.135989339791719, 'labSC': -0.006385206, 'predCH': 90.05623880461634, 'predSC': 0.11454602}
cluster num: 3
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[229.  92.   0. 158.]
 [177. 246.   0. 233.]
 [120. 128.   0. 145.]
 [248. 112.   0. 472.]]
val

In [4]:
kmeans_result = train_and_val_Kmeans(4, train_feas, train_labels, val_feas, val_labels)

training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[232.  63.  85.  99.]
 [189. 216.  91. 160.]
 [118. 107.  59. 109.]
 [285.  90.  85. 372.]]
val confusion matrix
[[ 49.  14.  32.  26.]
 [ 39.  54.  22.  39.]
 [ 40.  17.  22.  23.]
 [ 71.  16.  24. 102.]]
train result:
{'NMI': 0.036612529831182176, 'ARI': 0.040362952179764794, 'ACC': 0.37245762711864405, 'labCH': 18.484698013772995, 'labSC': -0.0074504963, 'predCH': 189.38140998817778, 'predSC': 0.06797423}
val result:
{'NMI': 0.05175919878311591, 'ARI': 0.05340456843617161, 'ACC': 0.3847457627118644, 'labCH': 6.135989339791719, 'labSC': -0.006385206, 'predCH': 48.049759476797455, 'predSC': 0.063885756}


In [3]:
task='byol'
model_type='PointSwin'
point2img=False
opti='adamw'
net, train_feas, train_labels, val_feas, val_labels = getPretrainModelandFeatures(task=task, 
                                                                                  model_type=model_type, 
                                                                                  point2img=point2img, 
                                                                                  opti=opti,
                                                                                  load_pretrain=True)
for clsnum in clusters_num:
    print('='*20)
    print('cluster num: {}'.format(clsnum))
    kmeans_result = train_and_val_Kmeans(clsnum, train_feas, train_labels, val_feas, val_labels)

Info Config:
{'dataset_path': './data', 'result_root': './result/Clustering', 'task': 'byol', 'train_db_name': 'tube', 'val_db_name': 'tube', 'BackboneType': 'PointSwin'}
Backbone Config:
{'BackboneConfig': {'in_channels': 15, 'feature_dim': 128, 'hiddim': 64, 'heads': 8, 'headdim': 32, 'embeddim': 4, 'stage_num': 3, 'downscale': 4, 'layer_num': 1, 'window_size': 4, 'attnlayer': 1}, 'BackboneType': 'PointSwin', 'num_cluster': 4, 'task': 'byol', 'head': 'mlp', 'nheads': 1, 'contrastive_feadim': 128}
Get backbone


  0%|                                                                                          | 0/295 [00:00<?, ?it/s]

Restart from checkpoint ./result/Clustering\tube\PointSwin\pretext\byol\byol\
Get dataset and dataloaders
get train features:


  0%|                                                                                           | 0/74 [00:00<?, ?it/s]

feature shape: (2360, 128), labels shape: (2360,)
get val features:


100%|██████████████████████████████████████████████████████████████████████████████████| 74/74 [00:09<00:00,  7.91it/s]


feature shape: (590, 128), labels shape: (590,)
cluster num: 2
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[  0. 190.   0. 290.]
 [  0. 497.   0. 137.]
 [  0. 248.   0. 164.]
 [  0. 111.   0. 723.]]
val confusion matrix
[[  0.  49.   0.  71.]
 [  0. 134.   0.  42.]
 [  0.  49.   0.  34.]
 [  0.  20.   0. 191.]]
train result:
{'NMI': 0.1512596698657848, 'ARI': 0.17865618089567967, 'ACC': 0.5169491525423728, 'labCH': 362.23629160217445, 'labSC': -0.00045375823, 'predCH': 10969.664004987852, 'predSC': 0.7353647}
val result:
{'NMI': 0.17277574575078783, 'ARI': 0.2050503634385759, 'ACC': 0.5508474576271186, 'labCH': 96.7489482228953, 'labSC': 0.03433136, 'predCH': 3332.8508505651243, 'predSC': 0.7635255}
cluster num: 3
training Kmeans model...
get matches
Assigning pred...
Counting confusion matrix...
train confusion matrix:
[[  0. 167.  75. 238.]
 [  0. 469.  79.  86.]
 [  0. 222.  99.  91.]
 [  0.  89. 105. 640.]]
val confus